In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
#from sklearn.model_selection import train_test_split
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from xgboost import XGBRegressor
import lightgbm as lgb
from catboost import CatBoostClassifier
#from sklearn.linear_model import LinearRegression
import optuna

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# check on the panda version and its dependencies
# i run this from time to time to ensure all is up to date
pd.__version__
#pd.show_versions()

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
# detect number of folds
fold_no = df_train['kfolds'].max() +1
fold_no

In [ ]:
df_train.head()

In [ ]:
# looking at f_27

In [ ]:
# all strings in f_27 are length 10 in train and test data

# df_train['f_27_len'] = df_train['f_27'].astype(str).map(len)
# df_test['f_27_len'] = df_test['f_27'].astype(str).map(len)
# df_train['f_27_len'].unique().tolist(), df_test['f_27_len'].unique().tolist()

In [ ]:
# label encode

f27_col_list = []
for i in range(1,11):
    df_train['f_27_' + str(i)] = df_train['f_27'].str.split('',expand=True)[i]
    df_test['f_27_' + str(i)] = df_test['f_27'].str.split('',expand=True)[i]
    f27_col_list.append('f_27_' + str(i))

In [ ]:
df_train

In [ ]:
f27_col_list

In [ ]:
for col in f27_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
unique_f27=[]
unique_f27_test=[]


for item in df_train['f_27']:
    unique_f27.append(len(set(item)))
    
for item in df_test['f_27']:
    unique_f27_test.append(len(set(item)))
    

df_train['unique_f27'] = unique_f27
df_test['unique_f27'] = unique_f27_test

In [ ]:
# more label encoding of f_29 and f_30

df_train['f_30_0'] = np.where(df_train['f_30'] == 0, 1, 0)
df_train['f_30_1'] = np.where(df_train['f_30'] == 1, 1, 0) 
df_train['f_30_2'] = np.where(df_train['f_30'] == 2, 1, 0) 
df_test['f_30_0'] = np.where(df_test['f_30'] == 0, 1, 0)
df_test['f_30_1'] = np.where(df_test['f_30'] == 1, 1, 0) 
df_test['f_30_2'] = np.where(df_test['f_30'] == 2, 1, 0) 

df_train['f_29_0'] = np.where(df_train['f_29'] == 0, 1, 0)
df_train['f_29_1'] = np.where(df_train['f_29'] == 1, 1, 0) 
df_test['f_29_0'] = np.where(df_test['f_29'] == 0, 1, 0)
df_test['f_29_1'] = np.where(df_test['f_29'] == 1, 1, 0) 


In [ ]:
f29_col_list=['f_29_0','f_29_1']

for col in f29_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
f30_col_list=['f_30_0','f_30_1','f_30_2']

for col in f30_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
continuous_feat = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19',
                   'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']

def stat_features(df, cols = continuous_feat):
    '''
    Calculate aggregated features across the selected continuous columns
    
    '''
    # Base statistical features.
    df['f_sum']  = df[continuous_feat].sum(axis=1)
    df['f_min']  = df[continuous_feat].min(axis=1)
    df['f_max']  = df[continuous_feat].max(axis=1)
    df['f_std']  = df[continuous_feat].std(axis=1)    
    df['f_mad']  = df[continuous_feat].mad(axis=1)
    df['f_mean'] = df[continuous_feat].mean(axis=1)
    df['f_kurt'] = df[continuous_feat].kurt(axis=1)

    # Extra statistical features.
    df['f_prod'] = df[continuous_feat].prod(axis=1)
    df['f_range'] = df[continuous_feat].max(axis=1) - df[continuous_feat].min(axis=1)
    df['f_count_pos']  = df[df[continuous_feat].gt(0)].count(axis=1)
    df['f_count_neg']  = df[df[continuous_feat].lt(0)].count(axis=1)

    return df

In [ ]:
df_train = stat_features(df_train, continuous_feat)
df_test = stat_features(df_test, continuous_feat)

In [ ]:
df_train

In [ ]:
# rename the nominal columns
# list of nominal columns will be where there are less than 20 unique numers

nom_cols = []

for i in range(len(df_train.columns)):
    if df_train[df_train.columns[i]].nunique() < 30:
        nom_cols.append(df_train.columns[i])

# remove 'target'
nom_cols.remove('target')
nom_cols.remove('kfolds')

print(nom_cols)

In [ ]:
# put the rest of the columns in another list to scale
# and remove f_27

all_cols = df_train.columns.tolist()

rest_cols = list(set(all_cols) - set(nom_cols))
rest_cols.remove('target')
rest_cols.remove('id')
rest_cols.remove('kfolds')
rest_cols.remove('f_27')

print(rest_cols)

In [ ]:
# scale the non integer columns

scaler = RobustScaler()
df_train[rest_cols] = scaler.fit_transform(df_train[rest_cols])
df_test[rest_cols] = scaler.transform(df_test[rest_cols]) 

In [ ]:
df_train.head()

In [ ]:
useful_features = [c for c in all_cols if c not in ('id','target','kfolds','f_27','f_29','f_30')]
df_test = df_test[useful_features]
print(useful_features)

In [ ]:
# taken from the hypertuning notebook https://www.kaggle.com/code/andrewnuk/hypertuning-xgb-cb-lgbm

params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'tree_method': 'hist',
    'use_label_encoder': False,
    'n_estimators': 10000,
    'learning_rate': 0.04663877705818151,
    'reg_lambda': 0.0009264101732567401,
    'reg_alpha': 0.0243021228407322,
    'subsample': 0.45210345718351885,
    'colsample_bytree': 0.9904612423331298,
    'max_depth': 6}

In [ ]:
final_predictions = []
final_valid_predictions = {}
scores = []

for fold in range(fold_no):
    xtrain =  df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
#     xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
#     model = XGBRegressor(random_state=0, n_jobs=6) # i have 8 cores but want to keep 2 open
    
    model = XGBRegressor(random_state=0, n_jobs=-1, **params)
       
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    #rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    roc = roc_auc_score(yvalid, preds_valid)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores), np.std(scores))

In [ ]:
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("train_pred_1_FE_20220524.csv", index=False)

df_sampleSubmission.target = np.mean(np.column_stack(final_predictions), axis=1)
df_sampleSubmission.columns = ["id", "pred_1"]
df_sampleSubmission.to_csv("test_pred_1_FE_20220524.csv", index=False)

In [ ]:
# column_names = ["id", "target"]

# # df_sampleSubmission.target = np.mean(np.column_stack(final_predictions), axis=1)
# df_sampleSubmission.to_csv("submission20220524a.csv", header=column_names, index=False)

In [ ]:
df_sampleSubmission.head()

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
df_train.head()

In [ ]:
# label encode

f27_col_list = []
for i in range(1,11):
    df_train['f_27_' + str(i)] = df_train['f_27'].str.split('',expand=True)[i]
    df_test['f_27_' + str(i)] = df_test['f_27'].str.split('',expand=True)[i]
    f27_col_list.append('f_27_' + str(i))

In [ ]:
df_train

In [ ]:
f27_col_list

In [ ]:
for col in f27_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
unique_f27=[]
unique_f27_test=[]


for item in df_train['f_27']:
    unique_f27.append(len(set(item)))
    
for item in df_test['f_27']:
    unique_f27_test.append(len(set(item)))
    

df_train['unique_f27'] = unique_f27
df_test['unique_f27'] = unique_f27_test

In [ ]:
# more label encoding of f_29 and f_30

df_train['f_30_0'] = np.where(df_train['f_30'] == 0, 1, 0)
df_train['f_30_1'] = np.where(df_train['f_30'] == 1, 1, 0) 
df_train['f_30_2'] = np.where(df_train['f_30'] == 2, 1, 0) 
df_test['f_30_0'] = np.where(df_test['f_30'] == 0, 1, 0)
df_test['f_30_1'] = np.where(df_test['f_30'] == 1, 1, 0) 
df_test['f_30_2'] = np.where(df_test['f_30'] == 2, 1, 0) 

df_train['f_29_0'] = np.where(df_train['f_29'] == 0, 1, 0)
df_train['f_29_1'] = np.where(df_train['f_29'] == 1, 1, 0) 
df_test['f_29_0'] = np.where(df_test['f_29'] == 0, 1, 0)
df_test['f_29_1'] = np.where(df_test['f_29'] == 1, 1, 0) 


In [ ]:
f29_col_list=['f_29_0','f_29_1']

for col in f29_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
f30_col_list=['f_30_0','f_30_1','f_30_2']

for col in f30_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
continuous_feat = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19',
                   'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']

def stat_features(df, cols = continuous_feat):
    '''
    Calculate aggregated features across the selected continuous columns
    
    '''
    # Base statistical features.
    df['f_sum']  = df[continuous_feat].sum(axis=1)
    df['f_min']  = df[continuous_feat].min(axis=1)
    df['f_max']  = df[continuous_feat].max(axis=1)
    df['f_std']  = df[continuous_feat].std(axis=1)    
    df['f_mad']  = df[continuous_feat].mad(axis=1)
    df['f_mean'] = df[continuous_feat].mean(axis=1)
    df['f_kurt'] = df[continuous_feat].kurt(axis=1)

    # Extra statistical features.
    df['f_prod'] = df[continuous_feat].prod(axis=1)
    df['f_range'] = df[continuous_feat].max(axis=1) - df[continuous_feat].min(axis=1)
    df['f_count_pos']  = df[df[continuous_feat].gt(0)].count(axis=1)
    df['f_count_neg']  = df[df[continuous_feat].lt(0)].count(axis=1)

    return df

In [ ]:
df_train = stat_features(df_train, continuous_feat)
df_test = stat_features(df_test, continuous_feat)

In [ ]:
df_train

In [ ]:
# rename the nominal columns
# list of nominal columns will be where there are less than 20 unique numers

nom_cols = []

for i in range(len(df_train.columns)):
    if df_train[df_train.columns[i]].nunique() < 30:
        nom_cols.append(df_train.columns[i])

# remove 'target'
nom_cols.remove('target')
nom_cols.remove('kfolds')

print(nom_cols)

In [ ]:
# put the rest of the columns in another list to scale
# and remove f_27

all_cols = df_train.columns.tolist()

rest_cols = list(set(all_cols) - set(nom_cols))
rest_cols.remove('target')
rest_cols.remove('id')
rest_cols.remove('kfolds')
rest_cols.remove('f_27')

print(rest_cols)

In [ ]:
# scale the non integer columns

scaler = RobustScaler()
df_train[rest_cols] = scaler.fit_transform(df_train[rest_cols])
df_test[rest_cols] = scaler.transform(df_test[rest_cols]) 

In [ ]:
df_train.head()

In [ ]:
useful_features = [c for c in all_cols if c not in ('id','target','kfolds','f_27','f_29','f_30')]
df_test = df_test[useful_features]
print(useful_features)

In [ ]:
# taken from the hypertuning notebook https://www.kaggle.com/code/andrewnuk/hypertuning-xgb-cb-lgbm

params_cb = {
    'loss_function': 'CrossEntropy',
    'eval_metric': 'AUC',
    'bootstrap_type': 'Bernoulli',
    'n_estimators': 10000,
    'learning_rate': 0.21871589947723546,
    'l2_leaf_reg': 3.89575022544331,
    'min_data_in_leaf': 90,
    'depth': 7,
    'leaf_estimation_iterations': 8,
    'subsample': 0.692447725389005}


In [ ]:
final_predictions = []
final_valid_predictions = {}
scores = []

for fold in range(fold_no):
    xtrain =  df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()    

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
#     xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
#     model = XGBRegressor(random_state=0, n_jobs=6) # i have 8 cores but want to keep 2 open
    
    model = CatBoostClassifier(random_state=0,  **params_cb)
       
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict_proba(xvalid)[:, -1]
    test_preds = model.predict_proba(xtest)[:, -1]
    final_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))    
    #rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    roc = roc_auc_score(yvalid, preds_valid)
    print(fold, roc)
    scores.append(roc)
   
    
print(np.mean(scores), np.std(scores))

In [ ]:
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("train_pred_2_FE_20220524.csv", index=False)

df_sampleSubmission.target = np.mean(np.column_stack(final_predictions), axis=1)
df_sampleSubmission.columns = ["id", "pred_2"]
df_sampleSubmission.to_csv("test_pred_2_FE_20220524.csv", index=False)

In [ ]:
# column_names = ["id", "target"]

# # df_sampleSubmission.target = np.mean(np.column_stack(final_predictions), axis=1)
# df_sampleSubmission.to_csv("submission20220524b.csv", header=column_names, index=False)

In [ ]:
df_sampleSubmission.head()

In [ ]:
# kfolds-5/train_folds_5.csv is an output from https://www.kaggle.com/code/andrewnuk/kfolds-5

df_train = pd.read_csv('/kaggle/input/kfolds-5/train_folds_5.csv')
df_test = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/test.csv')
df_sampleSubmission = pd.read_csv('/kaggle/input/tabular-playground-series-may-2022/sample_submission.csv')

In [ ]:
df_train.head()

In [ ]:
# label encode

f27_col_list = []
for i in range(1,11):
    df_train['f_27_' + str(i)] = df_train['f_27'].str.split('',expand=True)[i]
    df_test['f_27_' + str(i)] = df_test['f_27'].str.split('',expand=True)[i]
    f27_col_list.append('f_27_' + str(i))

In [ ]:
df_train

In [ ]:
f27_col_list

In [ ]:
for col in f27_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
unique_f27=[]
unique_f27_test=[]


for item in df_train['f_27']:
    unique_f27.append(len(set(item)))
    
for item in df_test['f_27']:
    unique_f27_test.append(len(set(item)))
    

df_train['unique_f27'] = unique_f27
df_test['unique_f27'] = unique_f27_test

In [ ]:
# more label encoding of f_29 and f_30

df_train['f_30_0'] = np.where(df_train['f_30'] == 0, 1, 0)
df_train['f_30_1'] = np.where(df_train['f_30'] == 1, 1, 0) 
df_train['f_30_2'] = np.where(df_train['f_30'] == 2, 1, 0) 
df_test['f_30_0'] = np.where(df_test['f_30'] == 0, 1, 0)
df_test['f_30_1'] = np.where(df_test['f_30'] == 1, 1, 0) 
df_test['f_30_2'] = np.where(df_test['f_30'] == 2, 1, 0) 

df_train['f_29_0'] = np.where(df_train['f_29'] == 0, 1, 0)
df_train['f_29_1'] = np.where(df_train['f_29'] == 1, 1, 0) 
df_test['f_29_0'] = np.where(df_test['f_29'] == 0, 1, 0)
df_test['f_29_1'] = np.where(df_test['f_29'] == 1, 1, 0) 


In [ ]:
f29_col_list=['f_29_0','f_29_1']

for col in f29_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
f30_col_list=['f_30_0','f_30_1','f_30_2']

for col in f30_col_list:
    df_train[col] = LabelEncoder().fit_transform(df_train[col])
    df_test[col] = LabelEncoder().fit_transform(df_test[col])

In [ ]:
df_train

In [ ]:
continuous_feat = ['f_00', 'f_01', 'f_02', 'f_03', 'f_04', 'f_05', 'f_06', 'f_19',
                   'f_20', 'f_21', 'f_22', 'f_23', 'f_24', 'f_25', 'f_26', 'f_28']

def stat_features(df, cols = continuous_feat):
    '''
    Calculate aggregated features across the selected continuous columns
    
    '''
    # Base statistical features.
    df['f_sum']  = df[continuous_feat].sum(axis=1)
    df['f_min']  = df[continuous_feat].min(axis=1)
    df['f_max']  = df[continuous_feat].max(axis=1)
    df['f_std']  = df[continuous_feat].std(axis=1)    
    df['f_mad']  = df[continuous_feat].mad(axis=1)
    df['f_mean'] = df[continuous_feat].mean(axis=1)
    df['f_kurt'] = df[continuous_feat].kurt(axis=1)

    # Extra statistical features.
    df['f_prod'] = df[continuous_feat].prod(axis=1)
    df['f_range'] = df[continuous_feat].max(axis=1) - df[continuous_feat].min(axis=1)
    df['f_count_pos']  = df[df[continuous_feat].gt(0)].count(axis=1)
    df['f_count_neg']  = df[df[continuous_feat].lt(0)].count(axis=1)

    return df

In [ ]:
df_train = stat_features(df_train, continuous_feat)
df_test = stat_features(df_test, continuous_feat)

In [ ]:
# rename the nominal columns
# list of nominal columns will be where there are less than 20 unique numers

nom_cols = []

for i in range(len(df_train.columns)):
    if df_train[df_train.columns[i]].nunique() < 30:
        nom_cols.append(df_train.columns[i])

# remove 'target'
nom_cols.remove('target')
nom_cols.remove('kfolds')

print(nom_cols)

In [ ]:
# put the rest of the columns in another list to scale
# and remove f_27

all_cols = df_train.columns.tolist()

rest_cols = list(set(all_cols) - set(nom_cols))
rest_cols.remove('target')
rest_cols.remove('id')
rest_cols.remove('kfolds')
rest_cols.remove('f_27')

print(rest_cols)

In [ ]:
# scale the non integer columns

scaler = RobustScaler()
df_train[rest_cols] = scaler.fit_transform(df_train[rest_cols])
df_test[rest_cols] = scaler.transform(df_test[rest_cols]) 

In [ ]:
df_train.head()

In [ ]:
useful_features = [c for c in all_cols if c not in ('id','target','kfolds','f_27','f_29','f_30')]
df_test = df_test[useful_features]
print(useful_features)

In [ ]:
# taken from the hypertuning notebook https://www.kaggle.com/code/andrewnuk/hypertuning-xgb-cb-lgbm

params_lgb = {
    'objective': 'binary',
    'metric': 'auc',
    'n_estimators': 20000,
    'learning_rate': 0.03094384584849164,
    'reg_lambda': 0.016917918416652716,
    'reg_alpha': 1.828457337138212e-05,
    'subsample': 0.8975450099423059,
    'subsample_freq': 6,
    'colsample_bytree': 0.9900145862697067,
    'min_child_weight': 65,
    'min_child_samples': 23}

In [ ]:
final_predictions = []
final_valid_predictions = {}
scores = []

for fold in range(fold_no):
    xtrain =  df_train[df_train['kfolds'] != fold].reset_index(drop=True)
    xvalid = df_train[df_train['kfolds'] == fold].reset_index(drop=True)
    xtest = df_test.copy()
    
    valid_ids = xvalid.id.values.tolist()    

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
#     ordinal_encoder = preprocessing.OrdinalEncoder()
#     xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
#     xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
#     xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
#     model = XGBRegressor(random_state=0, n_jobs=6) # i have 8 cores but want to keep 2 open
    
    model = lgb.LGBMClassifier(random_state=0, n_jobs=-1, **params_lgb)
       
    model.fit(xtrain, ytrain, early_stopping_rounds=1000, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict_proba(xvalid)[:, -1]
    test_preds = model.predict_proba(xtest)[:, -1]
    final_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))    
    #rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    roc = roc_auc_score(yvalid, preds_valid)
    print(fold, roc)
    scores.append(roc)

print(np.mean(scores), np.std(scores))

In [ ]:
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("train_pred_3_FE_20220524.csv", index=False)

df_sampleSubmission.target = np.mean(np.column_stack(final_predictions), axis=1)
df_sampleSubmission.columns = ["id", "pred_3"]
df_sampleSubmission.to_csv("test_pred_3_FE_20220524.csv", index=False)

In [ ]:
# column_names = ["id", "target"]

# # df_sampleSubmission.target = np.mean(np.column_stack(final_predictions), axis=1)
# df_sampleSubmission.to_csv("submission20220524c.csv", header=column_names, index=False)

In [ ]:
df_sampleSubmission.head()